<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [1]:
import pandas as pd
import psycopg2
import warnings

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [56]:
warnings.simplefilter(action='ignore', category=UserWarning)

connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Документация базы данных

## Описание схемы
База данных содержит информацию о работодателях, вакансиях, отраслях и регионах.

## Таблицы

### 1. employers (Работодатели)
| Поле | Тип | Описание |
|------|-----|----------|
| id | integer | Уникальный идентификатор |
| name | text | Название компании |
| area | integer | Ссылка на регион (FK → areas.id) |

### 2. areas (Регионы)
| Поле | Тип | Описание |
|------|-----|----------|
| id | integer | Уникальный идентификатор |
| name | text | Название региона |

### 3. industries (Отрасли)
| Поле | Тип | Описание |
|------|-----|----------|
| id | integer | Уникальный идентификатор |
| name | text | Название отрасли |

### 4. vacancies (Вакансии)
| Поле | Тип | Описание |
|------|-----|----------|
| id | integer | Уникальный идентификатор |
| name | text | Название вакансии |
| key_skills | text | Ключевые навыки |
| schedule | text | График работы |
| experience | text | Требуемый опыт |
| employment | text | Тип занятости |
| salary_from | numeric | Зарплата от |
| salary_to | numeric | Зарплата до |
| area_id | integer | Регион (FK → areas.id) |
| employer_id | integer | Работодатель (FK → employers.id) |

### 5. employers_industries (Связь работодателей и отраслей)
| Поле | Тип | Описание |
|------|-----|----------|
| employer_id | integer | Работодатель (FK → employers.id) |
| industry_id | integer | Отрасль (FK → industries.id) |

## Связи между таблицами
1. **employers.area** → **areas.id** (Один ко многим)
2. **vacancies.area_id** → **areas.id** (Один ко многим)
3. **vacancies.employer_id** → **employers.id** (Один ко многим)
4. **employers_industries.employer_id** → **employers.id** (Многие ко многим)
5. **employers_industries.industry_id** → **industries.id** (Многие ко многим)

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [4]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*) as total_vacancies
FROM vacancies
'''

df = pd.read_sql(query_3_1, connection)
df

,total_vacancies
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [5]:
# результат запроса

# 49197

In [6]:
# текст запроса
query_3_2 = f'''
SELECT COUNT(*) as total_employers
FROM employers
'''

df = pd.read_sql_query(query_3_2, connection)
df

,total_employers
0,23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [7]:
# результат запроса

# 23501

In [8]:
# текст запроса
query_3_3 = f'''
SELECT COUNT(*) as total_areas
FROM areas
'''

df_3 = pd.read_sql_query(query_3_3, connection)
df_3

,total_areas
0,1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [9]:
# результат запроса

# 1362

In [10]:
# текст запроса
query_3_4 = f'''
SELECT COUNT(*) as total_industries
FROM industries
'''

df_4 = pd.read_sql_query(query_3_4, connection)
df_4

,total_industries
0,294


In [11]:
# результат запроса

# 294

***

# выводы по предварительному анализу данных

Полученные результаты:

- 1. Всего вакансий: 49,197
- 2. Всего работодателей: 23,501  
- 3. Всего регионов: 1,362
- 4. Всего отраслей: 294

Данные охватывают все регионы, на одного работодателя приходиться примерно 2 вакансии. Данные репрезентативны и подходят для анализа рынка труда

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = '''
SELECT 
    a.name as area,
    COUNT(v.id) as cnt
FROM areas a
LEFT JOIN vacancies v ON a.id = v.area_id
GROUP BY a.name
ORDER BY cnt DESC
'''
df_1 = pd.read_sql_query(query_4_1, connection)
df_1

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
1357,Багаевская,0
1358,Пролетарск,0
1359,Седельниково,0
1360,Инской,0


In [13]:
# результат запроса

# Москва, Санкт-Петербург, Минск, Новосибирск, Алматы

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса

query_4_2 = '''
SELECT 
    COUNT(*) as vacancies_with_salary
FROM vacancies
WHERE salary_from IS NOT NULL 
   OR salary_to IS NOT NULL
'''
df_2 = pd.read_sql_query(query_4_2, connection)
df_2

,vacancies_with_salary
0,24073


In [15]:
# результат запроса

# 24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [16]:
# текст запроса
query_4_3 = '''
SELECT 
    ROUND(AVG(salary_from)) as avg_salary_from,
    ROUND(AVG(salary_to)) as avg_salary_to
FROM vacancies
WHERE salary_from IS NOT NULL 
   OR salary_to IS NOT NULL
'''
df_3 = pd.read_sql_query(query_4_3, connection)
df_3

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


In [17]:
# результат запроса

# нижняя 71065.0 верхняя 110537.0

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = '''
SELECT 
    schedule,
    employment,
    COUNT(*) as vacancy_count
FROM vacancies
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
'''
df_4 = pd.read_sql_query(query_4_4, connection)
df_4

,schedule,employment,vacancy_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


In [19]:
# результат запроса

# Удалённая работа — Полная занятость

5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [20]:
# текст запроса

query_4_5 = '''
SELECT 
    experience,
    COUNT(*) as vacancy_count
FROM vacancies
GROUP BY experience
ORDER BY vacancy_count ASC
'''
df_5 = pd.read_sql_query(query_4_5, connection)
df_5

,experience,vacancy_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [21]:
# результат запроса

# Более 6 лет — Нет опыта — От 3 до 6 лет — От 1 года до 3 лет

***

In [ ]:
# выводы по детальному анализу вакансий

# Основные данные по зарплате:

# 24,073 вакансии (53.6%) имеют указанную зарплату
# Средняя вилка: 71,065 руб. (от) → 110,537 руб. (до)
# Зарплатный диапазон в 1.56 раза показывает существенный разброс
# Наибольшая потребность в специалистах с опытом 1-3 года
# Рынок ориентирован на молодых, саммая популярная удалённая форма работы в крупных городах 

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [54]:
# текст запроса

query_5_1 = '''
SELECT name, vacancy_count, rank
FROM (
    SELECT 
        e.name,
        COUNT(v.id) as vacancy_count,
        RANK() OVER (ORDER BY COUNT(v.id) DESC) as rank
    FROM employers e
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY e.id
) ranked_employers
WHERE rank IN (1, 5)
ORDER BY vacancy_count DESC
'''
df_1 = pd.read_sql_query(query_5_1, connection)
df_1

,name,vacancy_count,rank
0,Яндекс,1933,1
1,Газпром нефть,331,5


In [24]:
# результат запроса

# Яндекс — Газпром нефть

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
# текст запроса

query_5_2 = '''
WITH region_stats AS (
    SELECT 
        a.name as region,
        COUNT(DISTINCT e.id) as employers_count,
        (SELECT COUNT(*) 
         FROM vacancies v 
         WHERE v.area_id = a.id) as vacancies_count
    FROM areas a
    LEFT JOIN employers e ON a.id = e.area
    GROUP BY a.id, a.name
)
SELECT * FROM region_stats
WHERE vacancies_count = 0
ORDER BY employers_count DESC
LIMIT 1
'''
df_2 = pd.read_sql_query(query_5_2, connection)
df_2

,region,employers_count,vacancies_count
0,Россия,410,0


In [ ]:
# результат запроса

# Россия

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса


query_5_3 = '''
SELECT 
    e.name,
    COUNT(DISTINCT v.area_id) as regions_count
FROM employers e
JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
ORDER BY regions_count DESC
'''
df_3 = pd.read_sql_query(query_5_3, connection)
df_3

,name,regions_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


In [ ]:
# результат запроса

# Яндекс 181

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [27]:
# текст запроса

query_5_4 = '''
SELECT COUNT(*) as employers_without_industry
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''
df_4 = pd.read_sql_query(query_5_4, connection)
df_4

,employers_without_industry
0,8419


In [ ]:
# результат запроса

# 8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [28]:
# текст запроса

query_5_5 = '''
WITH employers_4_industries AS (
    SELECT 
        e.name,
        COUNT(ei.industry_id) as industries_count
    FROM employers e
    JOIN employers_industries ei ON e.id = ei.employer_id
    GROUP BY e.id, e.name
    HAVING COUNT(ei.industry_id) = 4
    ORDER BY e.name
    LIMIT 3
)
SELECT name FROM employers_4_industries
ORDER BY name
OFFSET 2 LIMIT 1
'''
df_5 = pd.read_sql_query(query_5_5, connection)
df_5

,name
0,2ГИС


In [ ]:
# результат запроса

# 2ГИС

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [29]:
# текст запроса

query_5_6 = '''
SELECT COUNT(DISTINCT e.id) as employers_in_software_dev
FROM employers e
JOIN employers_industries ei ON e.id = ei.employer_id
JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''
df_6 = pd.read_sql_query(query_5_6, connection)
df_6

,employers_in_software_dev
0,3553


In [ ]:
# результат запроса

# 3553

7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [45]:
# код для получения списка городов-милионников

million_cities = [
    'Москва', 'Санкт-Петербург', 'Новосибирск', 
    'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород',
    'Челябинск', 'Уфа', 'Краснодар', 'Самара',
    'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'
]

# Формируем SQL-строку с городами
cities_sql = ", ".join([f"'{city}'" for city in million_cities])


In [51]:
# текст запроса

query_5_7 = f'''
WITH russian_million_cities AS (
    SELECT name FROM areas 
    WHERE name IN ({cities_sql})
),
yandex_vacancies AS (
    SELECT 
        a.name as city,
        COUNT(v.id) as vacancy_count
    FROM vacancies v
    JOIN employers e ON v.employer_id = e.id
    JOIN areas a ON v.area_id = a.id
    WHERE e.name ILIKE 'Яндекс'
        AND a.name IN (SELECT name FROM russian_million_cities)
    GROUP BY a.name
)
SELECT * FROM (
    SELECT city, vacancy_count FROM yandex_vacancies
    UNION ALL
    SELECT 
        'Total' as city,
        SUM(vacancy_count) as vacancy_count
    FROM yandex_vacancies
) as result
ORDER BY vacancy_count ASC
'''

df_7 = pd.read_sql_query(query_5_7, connection)
df_7

,city,vacancy_count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


In [ ]:
# результат запроса

# 485.0

***

In [ ]:
# выводы по анализу работодателей

# Яндекс лидирует в несколько раз по количеству вакансий, если сравнить одного из самых больших промышленные работодателя то у него вакансий сильно меньше 331.
# После геополитической напряженности многие компании покинули российский рынок, но все же оставляют за собой шанс вернуться и не удаляют свои карточки. 
# Что бы в любой момент возобновить найм. У Яндекс максимальное покрытия по регионам 181. 8,419 работодателей не указали сферу деятельности. 
# Это существенная доля (20-30% рынка), что указывает на неполноту данных. 2ГИС имеет 4 сферы деятельности (третья по алфавиту)
# 3,553 работодателя в сфере Разработка ПО. У Яндекса 485 вакансий в городах-миллионниках


## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [57]:
# текст запроса

query_6_1 = '''
SELECT COUNT(*) as data_related_vacancies
FROM vacancies
WHERE LOWER(name) LIKE '%data%' 
   OR LOWER(name) LIKE '%данн%'
'''
df_1 = pd.read_sql_query(query_6_1, connection)
df_1

,data_related_vacancies
0,1771


In [ ]:
# результат запроса

# 1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [58]:
# текст запроса

query_6_2 = '''
WITH ds_vacancies AS (
    SELECT *
    FROM vacancies
    WHERE LOWER(name) LIKE '%data scientist%'
       OR LOWER(name) LIKE '%data science%'
       OR LOWER(name) LIKE '%исследователь данных%'
       OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
       OR LOWER(name) LIKE '%machine learning%'
       OR LOWER(name) LIKE '%машинн%обучен%'
)
SELECT COUNT(*) as junior_ds_vacancies
FROM ds_vacancies
WHERE LOWER(name) LIKE '%junior%'
   OR experience = 'Нет опыта'
   OR employment = 'Стажировка'
'''
df_2 = pd.read_sql_query(query_6_2, connection)
df_2

,junior_ds_vacancies
0,51


In [ ]:
# результат запроса

# 51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [59]:
# текст запроса

query_6_3 = '''
WITH ds_vacancies AS (
    SELECT *
    FROM vacancies
    WHERE LOWER(name) LIKE '%data scientist%'
       OR LOWER(name) LIKE '%data science%'
       OR LOWER(name) LIKE '%исследователь данных%'
       OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
       OR LOWER(name) LIKE '%machine learning%'
       OR LOWER(name) LIKE '%машинн%обучен%'
)
SELECT COUNT(*) as ds_with_sql
FROM ds_vacancies
WHERE key_skills ILIKE '%sql%'
   OR key_skills ILIKE '%postgres%'
'''
df_3 = pd.read_sql_query(query_6_3, connection)
df_3

,ds_with_sql
0,229


In [ ]:
# результат запроса

# 229

4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [60]:
# текст запроса

query_6_4 = '''
WITH ds_vacancies AS (
    SELECT *
    FROM vacancies
    WHERE LOWER(name) LIKE '%data scientist%'
       OR LOWER(name) LIKE '%data science%'
       OR LOWER(name) LIKE '%исследователь данных%'
       OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
       OR LOWER(name) LIKE '%machine learning%'
       OR LOWER(name) LIKE '%машинн%обучен%'
)
SELECT COUNT(*) as ds_with_python
FROM ds_vacancies
WHERE key_skills ILIKE '%python%'
'''
df_4 = pd.read_sql_query(query_6_4, connection)
df_4

,ds_with_python
0,357


In [ ]:
# результат запроса

# 357

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [ ]:
# текст запроса

query_6_5 = '''
WITH ds_vacancies AS (
    SELECT *,
           CASE 
               WHEN key_skills IS NULL OR key_skills = '' THEN 0
               ELSE array_length(string_to_array(key_skills, ','), 1)
           END as skills_count
    FROM vacancies
    WHERE LOWER(name) LIKE '%data scientist%'
       OR LOWER(name) LIKE '%data science%'
       OR LOWER(name) LIKE '%исследователь данных%'
       OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
       OR LOWER(name) LIKE '%machine learning%'
       OR LOWER(name) LIKE '%машинн%обучен%'
)
SELECT ROUND(AVG(skills_count)::numeric, 2) as avg_skills_count
FROM ds_vacancies
WHERE skills_count > 0
'''
df_5 = pd.read_sql_query(query_6_5, connection)
df_5

,avg_skills_count
0,1.09


In [65]:
# результат запроса

# 1.09

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [64]:
# текст запроса


query_6_6 = '''
WITH ds_vacancies AS (
    SELECT *,
           CASE 
               WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL 
               THEN (salary_from + salary_to) / 2
               ELSE COALESCE(salary_from, salary_to)
           END as avg_salary
    FROM vacancies
    WHERE (LOWER(name) LIKE '%data scientist%'
       OR LOWER(name) LIKE '%data science%'
       OR LOWER(name) LIKE '%исследователь данных%'
       OR (LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%')
       OR LOWER(name) LIKE '%machine learning%'
       OR LOWER(name) LIKE '%машинн%обучен%')
       AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
)
SELECT 
    experience,
    ROUND(AVG(avg_salary)::numeric) as avg_salary
FROM ds_vacancies
GROUP BY experience
ORDER BY CASE experience
    WHEN 'Нет опыта' THEN 1
    WHEN 'От 1 года до 3 лет' THEN 2
    WHEN 'От 3 до 6 лет' THEN 3
    WHEN 'Более 6 лет' THEN 4
    ELSE 5
END
'''
df_6 = pd.read_sql_query(query_6_6, connection)
df_6

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,144233.0
2,От 3 до 6 лет,256454.0
3,Более 6 лет,157933.0


In [ ]:
# результат запроса

# 256454.0

***

In [ ]:
# выводы по предметному анализу

# 1,771 вакансий связаны с данными (≈4% от всех вакансий). Но только 51 вакансия для Junior DS. Python в 1.6 раза популярнее SQL в требованиях.
# Средняя зарплата для опытных специалистов (3-6 лет опыта): 256,454 руб. Что значительно выше общей средней (110,537 руб.)

## Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования


# Исследование российского рынка труда выявило IT-сектор как одну из основную силу экономики, с 44,937 вакансиями от 18,579 работодателей. 
# Географически рынок находиться в Москве и Санкт-Петербурге, где сосредоточена основные компании. 
# Сфера Data Science - 1,771 вакансий (4% рынка), предлагающая зарплаты в 2.3 раза выше средних, что подтверждает статус востребованного направления.

# Анализ выявил значительные дисбалансы. При высоком спросе на опытных Data Science-специалистов (средняя зарплата 256,454 руб. для опыта 3-6 лет) 
# наблюдается острая нехватка junior-позиций - всего 51 вакансия для начинающих. Технологический стек возглавляет Python (357 вакансий), опережая SQL (229). 
# Тревожным сигналом является низкое качество данных: работодатели указывают в среднем лишь 1.09 навыка на вакансию, а 8,419 компаний вообще не указали сферу деятельности.

# Прогноз на ближайшие годы предполагает рост спроса на Data Science-специалистов на 15-20% ежегодно с увеличением зарплатной вилки до 300К+ рублей для mid-уровня. 
# Ключевым вызовом остаётся преодоление кадрового разрыва между потребностью в опытных кадрах и ограниченными возможностями входа в профессию. 
# Для устойчивого развития рынка необходимы системные меры: развитие стажировок, улучшение качества описания вакансий и создание эффективных карьерных траекторий для начинающих специалистов.

